# Train DL

> A collection of deep learning tools via Fastai

## Setup

In [1]:
#| default_exp dl

In [2]:
#| hide
import sys
sys.path.append("/notebooks/katlas")
from nbdev.showdoc import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
#| export
from fastbook import *
import fastcore.all as fc,torch.nn.init as init
from fastai.callback.training import GradientClip
from torch.utils.data import WeightedRandomSampler

# katlas
from katlas.core import *
from katlas.feature import *
from katlas.train import *

# sklearn
from sklearn.model_selection import *
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr,pearsonr

## Utils

In [4]:
#| export
def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [5]:
seed_everything()

## Load Data

In [6]:
# read training data
df = pd.read_parquet('train_data/combine_t5_kd.parquet').reset_index()

# read data contains info for split
info_df = Data.get_kinase_info_full().query('pseudo!="1"') # get non-pseudo kinase

# merge info with training data
info = df[['kinase']].merge(info_df)
info.head()

# splits
splits = get_splits(info,stratified='group')
split0 = splits[0]


# column name of feature and target
feat_col = df.columns[df.columns.str.startswith('T5_')]
target_col = df.columns[~df.columns.isin(feat_col)][1:]

StratifiedKFold(n_splits=5, random_state=123, shuffle=True)
# kinase group in train set: 9
# kinase group in test set: 9
---------------------------
# kinase in train set: 312
---------------------------
# kinase in test set: 78
---------------------------
test set: ['EPHA3' 'FES' 'FLT3' 'FYN' 'EPHB1' 'EPHB3' 'FER' 'EPHB4' 'FLT4' 'FGFR1' 'EPHA5' 'TEK' 'DDR2' 'ZAP70' 'LIMK1' 'ULK3' 'JAK1' 'WEE1' 'TESK1' 'MAP2K3' 'AMPKA2' 'ATM' 'CAMK1D' 'CAMK2D' 'CAMK4' 'CAMKK1'
 'CK1D' 'CK1E' 'DYRK2' 'DYRK4' 'HGK' 'IKKE' 'JNK2' 'JNK3' 'KHS1' 'MAPKAPK5' 'MEK2' 'MSK2' 'NDR1' 'NEK6' 'NEK9' 'NIM1' 'NLK' 'OSR1' 'P38A' 'P38B' 'P90RSK' 'PAK1' 'PERK' 'PKCH' 'PKCI' 'PKN1' 'ROCK2'
 'RSK2' 'SIK' 'STLK3' 'TAK1' 'TSSK1' 'ALPHAK3' 'BMPR2' 'CDK10' 'CDK13' 'CDK14' 'CDKL5' 'GCN2' 'GRK4' 'IRE1' 'KHS2' 'MASTL' 'MLK4' 'MNK1' 'MRCKA' 'PRPK' 'QSK' 'SMMLCK' 'SSTK' 'ULK2' 'VRK1']


## Dataset

In [7]:
#| export
class GeneralDataset:
    def __init__(self, 
                 df, # a dataframe of values
                 feat_col, # feature columns
                 target_col=None # Will return test set for prediction if target col is None
                ):
        "A general dataset that can be applied to any dataframe"
        
        self.test = False if target_col is not None else True
        
        self.X = df[feat_col].values 
        self.y = df[target_col].values if not self.test else None
        
        self.len = df.shape[0]

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        X = torch.Tensor(self.X[index])
        if self.test:
            return X
        else:
            y = torch.Tensor(self.y[index])
            return X, y

In [8]:
# dataset
ds = GeneralDataset(df,feat_col,target_col)

In [9]:
len(ds)

390

In [10]:
dl = DataLoader(ds, batch_size=64, shuffle=True)

In [11]:
#| export
def get_sampler(info,col):
    
    "For imbalanced data, get higher weights for less-represented samples"
    
    # get value counts
    group_counts = info[col].value_counts()
    
    # to reduce the difference through log
    # group_counts = group_counts.apply(lambda x: np.log(x+1.01))
    
    weights = 1. / group_counts[info[col]]

    sample_weights = torch.from_numpy(weights.to_numpy())
    sample_weights = torch.clamp_min(sample_weights,0.01)

    sampler = WeightedRandomSampler(sample_weights, len(sample_weights),replacement=True)
    
    return sampler

In [12]:
sampler = get_sampler(info,'subfamily')

In [13]:
# dataloader
dl = DataLoader(ds, batch_size=64, sampler=sampler)

In [14]:
xb,yb = next(iter(dl))

xb.shape,yb.shape

(torch.Size([64, 1024]), torch.Size([64, 210]))

## Models

### MLP

In [15]:
#| export
def MLP_1(num_features, 
          num_targets,
          hidden_units = [512, 218],
          dp = 0.2):
    
    # Start with the first layer from num_features to the first hidden layer
    layers = [
        nn.Linear(num_features, hidden_units[0]),
        nn.BatchNorm1d(hidden_units[0]),
        nn.Dropout(dp),
        nn.PReLU()
    ]
    
    # Loop over hidden units to create intermediate layers
    for i in range(len(hidden_units) - 1):
        layers.extend([
            nn.Linear(hidden_units[i], hidden_units[i+1]),
            nn.BatchNorm1d(hidden_units[i+1]),
            nn.Dropout(dp),
            nn.PReLU()
        ])
    
    # Add the output layer
    layers.append(nn.Linear(hidden_units[-1], num_targets))
    
    model = nn.Sequential(*layers)
    
    return model

In [16]:
n_feature = len(feat_col)
n_target = len(target_col)

In [17]:
model = MLP_1(n_feature, n_target)

In [18]:
model(xb)

tensor([[-0.1115, -0.3755, -0.3818,  ..., -0.1483, -0.0387, -0.1111],
        [ 0.8555,  0.9352, -0.9642,  ..., -0.4723,  0.7757, -0.0121],
        [ 0.3422,  0.3537, -0.1441,  ...,  0.5467, -0.4535,  0.2103],
        ...,
        [-0.4287,  0.6751,  0.1797,  ...,  0.0192,  0.0692, -0.0573],
        [-0.0206, -0.1953,  0.7445,  ..., -0.2206, -0.1188,  0.4579],
        [ 0.2342, -0.0243,  0.4630,  ...,  0.8393,  0.5747, -0.6881]], grad_fn=<AddmmBackward0>)

### CNN1D

***Version 1***

In [19]:
#| export
class CNN1D_1(Module):
    
    def __init__(self, 
                 num_features, # this does not matter, just for format
                 num_targets):

        self.conv1 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3, dilation=1, padding=1, stride=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels=3, out_channels=8, kernel_size=3, dilation=1, padding=1, stride=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.flatten = Flatten()
        self.fc1 = nn.Linear(in_features = int(8 * num_features/4), out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=num_targets)

    def forward(self, x):
        x = x.unsqueeze(1) # need shape (bs, 1, num_features) for CNN
        x = self.pool1(nn.functional.relu(self.conv1(x)))
        x = self.pool2(nn.functional.relu(self.conv2(x)))
        # x = torch.flatten(x, 1)
        x = self.flatten(x)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [20]:
model = CNN1D_1(n_feature, n_target)

In [21]:
model(xb)

tensor([[ 0.0193,  0.0690,  0.0138,  ..., -0.0428, -0.0026,  0.0840],
        [ 0.0203,  0.0693,  0.0136,  ..., -0.0422, -0.0023,  0.0846],
        [ 0.0198,  0.0703,  0.0148,  ..., -0.0424, -0.0029,  0.0839],
        ...,
        [ 0.0197,  0.0694,  0.0147,  ..., -0.0429, -0.0019,  0.0841],
        [ 0.0193,  0.0687,  0.0146,  ..., -0.0429, -0.0017,  0.0843],
        [ 0.0191,  0.0692,  0.0148,  ..., -0.0425, -0.0028,  0.0834]], grad_fn=<AddmmBackward0>)

***Version 2***

In [22]:
#| export
def init_weights(m, leaky=0.):
    "Initiate any Conv layer with Kaiming norm."
    if isinstance(m, (nn.Conv1d,nn.Conv2d,nn.Conv3d)): init.kaiming_normal_(m.weight, a=leaky)

In [23]:
#| export
def lin_wn(ni,nf,dp=0.1,act=nn.SiLU):
    "Weight norm of linear."
    layers =  nn.Sequential(
            nn.BatchNorm1d(ni),
            nn.Dropout(dp),
            nn.utils.weight_norm(nn.Linear(ni, nf)) )
    if act: layers.append(act())
    return layers

In [24]:
#| export
def conv_wn(ni, nf, ks=3, stride=1, padding=1, dp=0.1,act=nn.ReLU):
    "Weight norm of conv."
    layers =  nn.Sequential(
        nn.BatchNorm1d(ni),
        nn.Dropout(dp),
        nn.utils.weight_norm(nn.Conv1d(ni, nf, ks, stride, padding)) )
    if act: layers.append(act())
    return layers

In [25]:
#| export
class CNN1D_2(nn.Module):
    
    def __init__(self, ni, nf, amp_scale = 16):
        super().__init__()

        cha_1,cha_2,cha_3 = 256,512,512
        hidden_size = cha_1*amp_scale

        cha_po_1 = hidden_size//(cha_1*2)
        cha_po_2 = (hidden_size//(cha_1*4)) * cha_3
        
        self.lin = lin_wn(ni,hidden_size)
        
        # bs, 256, 16
        self.view = View(-1,cha_1,amp_scale)
        
        self.conv1 = nn.Sequential(
            conv_wn(cha_1, cha_2, ks=5, stride=1, padding=2, dp=0.1),
            nn.AdaptiveAvgPool1d(output_size = cha_po_1),
            conv_wn(cha_2, cha_2, ks=3, stride=1, padding=1, dp=0.1))
        
        self.conv2 = nn.Sequential(
            conv_wn(cha_2, cha_2, ks=3, stride=1, padding=1, dp=0.3),
            conv_wn(cha_2, cha_3, ks=5, stride=1, padding=2, dp=0.2))
        
        self.head = nn.Sequential(
            nn.MaxPool1d(kernel_size=4, stride=2, padding=1),
            nn.Flatten(),
            lin_wn(cha_po_2,nf,act=None) )


    def forward(self, x):
        # amplify features to 4096
        x = self.lin(x)
        
        # reshape to bs,256,16 for conv1d
        x = self.view(x) 

        x = self.conv1(x)
        
        x_s = x  # for skip connection (multiply)
        x = self.conv2(x)
        x = x * x_s

        # Final block
        x = self.head(x)

        return x

In [26]:
model = CNN1D_2(n_feature,n_target).apply(init_weights)

In [27]:
model(xb)

tensor([[-0.5740,  0.0151, -0.0819,  ...,  0.2636,  0.3405, -0.1404],
        [-0.6800,  0.5530, -0.0958,  ..., -0.3752, -0.6124,  0.7171],
        [ 0.4427, -0.3204, -0.3243,  ..., -0.2290,  0.1070,  0.1504],
        ...,
        [-0.3660, -0.2667, -0.6036,  ..., -0.3130,  0.5462, -0.0055],
        [ 0.4511,  0.6824,  0.8659,  ..., -0.0171,  0.2362, -0.3475],
        [-0.0746, -0.1699,  0.6895,  ...,  1.1522, -0.3472,  0.6422]], grad_fn=<AddmmBackward0>)

## DL Trainer

In [29]:
#| export
def train_dl(df, 
            feat_col, 
            target_col,
            split, # tuple of numpy array for split index
            model_func, # function to get pytorch model
             n_epoch = 4, # number of epochs
             bs = 32, # batch size
             lr = 1e-2, # will be useless if lr_find is True
            loss = mse, # loss function
            save = None, # models/{save}.pth
             sampler = None,
             lr_find=False, # if true, will use lr from lr_find
              ):
    "A DL trainer."
    
    train = df.loc[split[0]]
    valid = df.loc[split[1]]
    
    train_ds = GeneralDataset(train, feat_col, target_col)
    valid_ds = GeneralDataset(valid, feat_col, target_col)
    
    n_workers = fc.defaults.cpus

    if sampler is not None:
        
        train_dl = DataLoader(train_ds, batch_size=bs, sampler=sampler,num_workers=n_workers)
        valid_dl = DataLoader(valid_ds, batch_size=bs, sampler=sampler,num_workers=n_workers)
        
        dls = DataLoaders(train_dl, valid_dl)
        
    else:
        
        dls = DataLoaders.from_dsets(train_ds, valid_ds, bs=bs, num_workers=n_workers)
    
    model = model_func()
    
    learn = Learner(dls.cuda(), model.cuda(), loss, 
                    metrics= [PearsonCorrCoef(),SpearmanCorrCoef()],
                    cbs = [GradientClip(1.0)] # prevent overfitting
                   )
    
    if lr_find:
        # get learning rate
        lr = learn.lr_find()
        plt.show()
        plt.close()
        print(lr)

        
    print('lr in training is', lr)
    learn.fit_one_cycle(n_epoch,lr) #cbs = [SaveModelCallback(fname = 'best')] # save best model
    
    if save is not None:
        learn.save(save)
        
    pred,target = learn.get_preds()
    
    pred = pd.DataFrame(pred.detach().cpu().numpy(),index=valid.index,columns=target_col)
    target = pd.DataFrame(target.detach().cpu().numpy(),index=valid.index,columns=target_col)
    
    return target, pred

In [28]:
def get_model():
    return CNN1D_2(n_feature, n_target)

In [30]:
target, pred = train_dl(df, 
                        feat_col, 
                        target_col,
                        split0, 
                        get_model,
                        n_epoch=1,
                        lr = 1e-2,
                        save = 'test')

lr in training is 0.01


epoch,train_loss,valid_loss,pearsonr,spearmanr,time
0,2.194746,2.176958,-0.104578,-0.053141,00:03


In [31]:
score_each(target,pred)

overall MSE: 2.1770
Average Pearson: 0.2149 


(2.176958,
 0.21488270397776432,
       Pearson
 3   -0.442855
 8   -0.490345
 10  -0.401885
 19  -0.428557
 24  -0.383956
 ..        ...
 359 -0.127000
 361  0.005761
 366  0.095977
 367  0.335805
 373 -0.230842
 
 [78 rows x 1 columns])

## DL CV

In [32]:
#| export
@fc.delegates(train_dl)
def train_dl_cv(df, 
                feat_col, 
                target_col, 
                splits, # list of tuples
                model_func, # functions like lambda x: return MLP_1(num_feat, num_target)
                save:str=None,
                **kwargs
                ):
    
    OOF = []
    metrics = []
    
    for fold,split in enumerate(splits):

        print(f'------fold{fold}------')
        
        
        fname=None
        # save best model for each fold
        if save is not None:
            fname = f'{save}_fold{fold}'
        
        # train model
        target, pred = train_dl(df,feat_col,target_col, split, model_func ,save=fname,**kwargs)

        #------------get scores--------------
        # get score metrics
        mse, pearson_avg, _ = score_each(target,pred)
        
        # store metrics in a dictionary for the current fold
        fold_metrics = {
            'fold': fold,
            'mse': mse,
            'pearson_avg': pearson_avg
        }
        metrics.append(fold_metrics)

        OOF.append(pred)
        

    # Concatenate OOF from each fold to a new dataframe
    oof = pd.concat(OOF).sort_index()
    
    # Get metrics into a dataframe
    metrics = pd.DataFrame(metrics)
    
    return oof, metrics

In [33]:
def get_model():
    return CNN1D_2(n_feature, n_target)

In [34]:
oof,metrics = train_dl_cv(df,feat_col,target_col,splits,get_model,n_epoch=1,lr=3e-3)

------fold0------
lr in training is 0.003


epoch,train_loss,valid_loss,pearsonr,spearmanr,time
0,1.165076,0.997911,0.091948,0.058285,00:01


overall MSE: 0.9979
Average Pearson: 0.1634 
------fold1------
lr in training is 0.003


epoch,train_loss,valid_loss,pearsonr,spearmanr,time
0,1.180757,0.992539,0.102852,0.084205,00:01


overall MSE: 0.9925
Average Pearson: 0.1617 
------fold2------
lr in training is 0.003


...

overall MSE: 0.9872
Average Pearson: 0.2364 
------fold3------
lr in training is 0.003


epoch,train_loss,valid_loss,pearsonr,spearmanr,time
0,1.184666,1.001829,0.077155,0.047876,00:01


overall MSE: 1.0018
Average Pearson: 0.1415 
------fold4------
lr in training is 0.003


epoch,train_loss,valid_loss,pearsonr,spearmanr,time
0,1.178444,0.992547,0.109969,0.100576,00:01


overall MSE: 0.9925
Average Pearson: 0.2014 


In [35]:
metrics

,fold,mse,pearson_avg
0,0,0.997911,0.163423
1,1,0.992539,0.161654
2,2,0.987170,0.236363
3,3,1.001829,0.141464
4,4,0.992547,0.201375


In [36]:
metrics.pearson_avg.mean()

0.18085578818910147

In [37]:
target = df[target_col]
_,_,corr = score_each(target,oof)

overall MSE: 0.9944
Average Pearson: 0.1809 


In [38]:
corr

,Pearson
0,-0.183429
1,-0.178564
2,-0.225202
3,-0.117838
4,-0.153463
...,...
385,0.109792
386,0.269238
387,0.079601
388,0.063310


## DL Predict

In [39]:
#| export
def predict_dl(df, 
               feat_col, 
               target_col,
               model, # model architecture
               model_pth, # only name, not with .pth
              ):
    
    "Predict dataframe given a deep learning model"
    
    test_dset = GeneralDataset(df,feat_col)
    test_dl = DataLoader(test_dset,bs=512)
    
    
    learn = Learner(None, model.cuda(), loss_func=1)
    learn.load(model_pth)
    
    learn.model.eval()
    
    preds = []
    for data in test_dl:
        inputs = data.cuda()
        outputs = learn.model(inputs) #learn.model(x).sigmoid().detach().cpu().numpy()

        preds.append(outputs.detach().cpu().numpy())

    preds = np.concatenate(preds)
    preds = pd.DataFrame(preds,index=df.index,columns=target_col)

    return preds

In [40]:
test = df.loc[split0[1]]

In [41]:
pred = predict_dl(test,
                  feat_col,
                  target_col, 
                  model,'test')
pred

,-5P,-5G,-5A,-5C,-5S,-5T,-5V,-5I,-5L,-5M,-5F,-5Y,-5W,-5H,-5K,-5R,-5Q,-5N,-5D,-5E,-5s,-5t,-5y,-4P,-4G,-4A,-4C,-4S,-4T,-4V,-4I,-4L,-4M,-4F,-4Y,-4W,-4H,-4K,-4R,-4Q,-4N,-4D,-4E,-4s,-4t,-4y,-3P,-3G,-3A,-3C,-3S,-3T,-3V,-3I,-3L,-3M,-3F,-3Y,-3W,-3H,-3K,-3R,-3Q,-3N,-3D,-3E,-3s,-3t,-3y,-2P,-2G,-2A,-2C,-2S,-2T,-2V,-2I,-2L,-2M,-2F,-2Y,-2W,-2H,-2K,-2R,-2Q,-2N,-2D,-2E,-2s,-2t,-2y,-1P,-1G,-1A,-1C,-1S,-1T,-1V,-1I,-1L,-1M,-1F,-1Y,-1W,-1H,-1K,-1R,-1Q,-1N,-1D,-1E,-1s,-1t,-1y,1P,1G,1A,1C,1S,1T,1V,1I,1L,1M,1F,1Y,1W,1H,1K,1R,1Q,1N,1D,1E,1s,1t,1y,2P,2G,2A,2C,2S,2T,2V,2I,2L,2M,2F,2Y,2W,2H,2K,2R,2Q,2N,2D,2E,2s,2t,2y,3P,3G,3A,3C,3S,3T,3V,3I,3L,3M,3F,3Y,3W,3H,3K,3R,3Q,3N,3D,3E,3s,3t,3y,4P,4G,4A,4C,4S,4T,4V,4I,4L,4M,4F,4Y,4W,4H,4K,4R,4Q,4N,4D,4E,4s,4t,4y,0s,0t,0y
3,0.827471,0.100224,0.832046,-0.875782,-0.163213,0.154136,-0.493984,-0.652184,-1.139407,-0.545682,0.170386,0.504751,-0.254571,-0.666113,-1.793947,-0.720070,-0.281525,-1.130579,-0.628133,-0.831370,1.536831,0.433035,0.392849,0.522302,-1.906116,0.269045,0.260138,-0.449420,-0.219241,-0.081720,-0.265319,-1.150107,-0.426660,0.034230,1.186783,1.155887,-0.541298,-0.147764,-0.391457,-0.532191,0.390349,-0.117091,-0.544431,-0.817431,0.209603,0.319055,-0.668265,-0.089056,-0.267722,0.873506,0.363645,0.925665,0.198308,0.376128,-0.091183,0.800630,2.594943,0.942247,0.381557,0.378796,-2.007185,-3.102536,-0.451001,0.324614,-0.403457,0.617391,0.573955,1.461275,0.632500,-1.116702,0.067198,-1.525167,0.533372,0.449234,-0.630748,0.000188,-0.406005,-0.597513,-0.477576,0.444630,1.485165,0.240802,0.838691,-2.154814,-2.045017,-0.508767,-0.634132,1.832220,-0.143416,-0.000741,0.834225,0.820550,-1.006547,-0.146693,-1.026853,1.660720,1.229229,0.859188,-0.824103,0.100888,-0.246177,1.906062,0.474571,1.582593,0.505408,0.352167,-1.356897,1.309204,1.164532,1.135649,-0.438278,0.429841,-1.104952,-1.197532,-0.390957,-1.573889,0.285766,-1.436248,0.650060,-0.180125,1.094425,0.196023,0.670384,0.781686,2.145067,0.097861,1.975262,1.960139,0.132345,0.234397,-0.731525,-1.392358,-0.154904,-1.966068,-2.782531,-0.367288,0.420465,-1.334604,-0.415886,-0.817545,0.334413,-0.257776,0.588528,0.056089,0.686094,-0.851600,-1.388452,0.962025,0.140007,1.217812,0.419813,1.218859,-1.105019,0.978566,-1.334749,-1.337203,-1.361357,-1.715217,-1.594962,0.368009,-0.917286,-0.569305,-0.249074,-0.784541,-0.572350,-0.319030,-0.301318,-0.291523,0.361332,-0.995329,0.845143,0.657074,1.600054,0.465445,0.819773,-0.739234,0.897671,-0.272619,-1.261856,-0.711332,-0.608858,-0.219403,1.353145,-0.526272,0.668316,-0.127292,-0.938279,-0.849241,0.974044,0.501423,0.257699,-0.155385,-1.242730,-0.218370,0.742504,0.296676,0.603466,-0.135024,1.358856,0.841600,0.340465,-1.716797,-0.272171,-1.086941,-0.108759,-1.107707,-0.915058,0.144688,1.703715,-1.096712
8,0.917410,0.171234,0.785013,-0.774128,-0.305260,-0.036252,-0.523487,-0.520050,-1.101476,-0.484559,-0.136079,0.453532,-0.157610,-0.722576,-1.705487,-0.679175,-0.613452,-0.849034,-0.617619,-0.829236,1.325297,0.568228,0.258706,0.530302,-1.917059,0.337471,0.461158,-0.513729,-0.009432,0.032227,-0.443830,-1.060179,-0.573893,0.035590,1.081311,1.032675,-0.778144,-0.034111,-0.088034,-0.417073,0.476150,-0.208814,-0.773442,-0.821986,0.408884,0.496548,-0.434809,-0.302523,-0.496544,0.771924,0.573002,0.862004,0.326757,0.434013,-0.269816,0.741980,2.483163,0.947341,0.286297,0.282709,-1.931927,-2.781210,-0.400135,0.336440,-0.553943,0.472467,0.712215,1.524756,0.716660,-1.030302,-0.101940,-1.563474,0.541147,0.501969,-0.687778,0.004106,-0.320514,-0.775591,-0.591284,0.515990,1.292747,0.403782,0.881161,-2.134602,-1.845125,-0.745882,-0.667191,1.685047,-0.176739,0.191540,0.742452,0.930569,-0.830662,-0.155687,-0.839995,1.647398,1.435268,0.932009,-0.948017,0.152050,-0.512855,1.643587,0.623791,1.754626,0.770352,0.206880,-1.215789,1.259650,1.313971,1.006722,-0.772838,0.465858,-0.794970,-1.441722,-0.327376,-1.471885,0.179104,-1.116795,0.668015,-0.096735,1.281591,0.349625,0.418560,0.723943,2.113839,0.183780,1.841187,1.912386,0.146610,0.131635,-0.489430,-1.257894

In [42]:
score_each(test[target_col],pred)

overall MSE: 2.1770
Average Pearson: 0.2149 


(2.176958126296138,
 0.21488261204649625,
       Pearson
 3   -0.442855
 8   -0.490345
 10  -0.401885
 19  -0.428557
 24  -0.383956
 ..        ...
 359 -0.127000
 361  0.005761
 366  0.095977
 367  0.335805
 373 -0.230842
 
 [78 rows x 1 columns])

In [43]:
#| hide
import nbdev; nbdev.nbdev_export()